In [2]:


import xarray as xr
import xesmf
#from HCtFlood.kara import flood_kara
#import bottleneck
import os
import numpy as np
import cftime
#%%


In [3]:
os.chdir("/home/nma/HDD_data/vinay/archives/IOMOM5Op/")
#exp_dir = '/home/nma/mom/vinay/MOM6dev/prepro/obc/'
exp_dir = '/home/nma/HDD_data/vinay/archives/IOMOM5Op/'

In [4]:
os.chdir("/home/nma/HDD_data/vinay/archives/IOMOM5Op/")

uu = xr.open_mfdataset("*.ocean_uvel.nc")
vv = xr.open_mfdataset("*.ocean_vvel.nc")
salt = xr.open_mfdataset("*.ocean_salt.nc")
ssh = xr.open_mfdataset("*.ocean_diag.nc").eta_t
temp = xr.open_mfdataset("*.ocean_temp.nc")
grd_p = "/home/nma/mom/MOM6dev/exps/2012_bob_obc/INPUT/horizontal_grid.nc"
#grd_p = "/home/nma/mom/MOM6dev/prepro/obc/obcmar/20120101.ocean_sfc.nc"
grid = xr.open_dataset(grd_p)
sam_op = xr.open_dataset("/home/nma/HDD_data/vinay/archives/finals/20130101.ocean_sfc.nc")

ds = xr.merge([uu,vv,salt,ssh,temp])
startslice = cftime.DatetimeNoLeap(2012, 1, 1,0,0,0,0,-1,1)
endslice = cftime.DatetimeNoLeap(2014, 1, 1,0,0,0,0,-1,1)

ds_cut = ds.sel(time=slice(startslice,endslice))



<xarray.DataArray 'temp' (time: 730, st_ocean: 58, yt_ocean: 240, xt_ocean: 360)>
dask.array<getitem, shape=(730, 58, 240, 360), dtype=float32, chunksize=(515, 58, 240, 360), chunktype=numpy.ndarray>
Coordinates:
  * st_ocean  (st_ocean) float64 0.5 1.5 2.5 3.5 ... 3.625e+03 4.3e+03 5e+03
  * time      (time) object 2012-01-01 12:00:00 ... 2013-12-31 12:00:00
  * xt_ocean  (xt_ocean) float64 30.0 30.25 30.5 30.75 ... 119.2 119.5 119.8
  * yt_ocean  (yt_ocean) float64 -30.0 -29.75 -29.5 -29.25 ... 29.25 29.5 29.75
Attributes:
    long_name:      Potential temperature
    units:          degrees C
    valid_range:    [-10. 100.]
    cell_methods:   time: mean
    time_avg_info:  average_T1,average_T2,average_DT
    coordinates:    geolon_t geolat_t
    standard_name:  sea_water_potential_temperature

## Make Grid object

In [5]:


xgrid = grid.x[::2,::2]
ygrid = grid.y[::2,::2]

south = xr.Dataset()
#south['lon'] = grid['x'].isel(nyp=0)
#south['lat'] = grid['y'].isel(nyp=0)

south['lon'] = xgrid.isel(nyp=0)
south['lat'] = ygrid.isel(nyp=0)

#lats = np.ones(len(grid['xq'].values))*grid['yq'][0].values

#south['lon'] = grid['xq']
#south['lat'] = lats

south



<xarray.Dataset>
Dimensions:  (nxp: 301)
Dimensions without coordinates: nxp
Data variables:
    lon      (nxp) float64 ...
    lat      (nxp) float64 ...

In [47]:
west = xr.Dataset()


west['lon'] = xgrid.isel(nxp=0)[:50]
west['lat'] = ygrid.isel(nxp=0)[:50]

west

<xarray.Dataset>
Dimensions:  (nyp: 50)
Dimensions without coordinates: nyp
Data variables:
    lon      (nyp) float64 ...
    lat      (nyp) float64 ...

In [48]:


regrid_south = xesmf.Regridder(ds_cut.rename({'xu_ocean': 'lon', 'yu_ocean': 'lat'}), south, 'bilinear', 
                               locstream_out=True, periodic=False, filename='regrid_south.nc')



regrid_west = xesmf.Regridder(ds_cut.rename({'xu_ocean': 'lon', 'yu_ocean': 'lat'}), west, 'bilinear', 
                               locstream_out=True, periodic=False, filename='regrid_west.nc')



In [49]:
v_south = regrid_south(ds_cut['v'])
u_south = regrid_south(ds_cut['u'])
temp_south = regrid_south(ds_cut['temp'])
ssh_south = regrid_south(ds_cut['eta_t'])
salt_south = regrid_south(ds_cut['salt'])


v_west = regrid_west(ds_cut['v'])
u_west = regrid_west(ds_cut['u'])
temp_west = regrid_west(ds_cut['temp'])
ssh_west = regrid_west(ds_cut['eta_t'])
salt_west = regrid_west(ds_cut['salt'])


v_west

/home/nma/nma/miniconda3/envs/mom/lib/python3.8/site-packages/xesmf/frontend.py:594: UserWarning: Using dimensions ('yu_ocean', 'xu_ocean') from data variable v as the horizontal dimensions for the regridding.
  warnings.warn(
/home/nma/nma/miniconda3/envs/mom/lib/python3.8/site-packages/xesmf/frontend.py:594: UserWarning: Using dimensions ('yu_ocean', 'xu_ocean') from data variable u as the horizontal dimensions for the regridding.
  warnings.warn(
/home/nma/nma/miniconda3/envs/mom/lib/python3.8/site-packages/xesmf/frontend.py:594: UserWarning: Using dimensions ('yt_ocean', 'xt_ocean') from data variable temp as the horizontal dimensions for the regridding.
  warnings.warn(
/home/nma/nma/miniconda3/envs/mom/lib/python3.8/site-packages/xesmf/frontend.py:594: UserWarning: Using dimensions ('yt_ocean', 'xt_ocean') from data variable eta_t as the horizontal dimensions for the regridding.
  warnings.warn(
/home/nma/nma/miniconda3/envs/mom/lib/python3.8/site-packages/xesmf/frontend.py:594: 

<xarray.DataArray (time: 730, st_ocean: 58, nyp: 50)>
dask.array<getitem, shape=(730, 58, 50), dtype=float32, chunksize=(515, 58, 50), chunktype=numpy.ndarray>
Coordinates:
  * st_ocean  (st_ocean) float64 0.5 1.5 2.5 3.5 ... 3.625e+03 4.3e+03 5e+03
  * time      (time) object 2012-01-01 12:00:00 ... 2013-12-31 12:00:00
    lon       (nyp) float64 77.0 77.0 77.0 77.0 77.0 ... 77.0 77.0 77.0 77.0
    lat       (nyp) float64 4.0 4.076 4.152 4.228 ... 7.496 7.572 7.648 7.724
Dimensions without coordinates: nyp
Attributes:
    regrid_method:  bilinear

In [50]:
drowned_v_south = v_south.ffill(dim='nxp').ffill(dim='st_ocean')
drowned_u_south =u_south.ffill(dim='nxp').ffill(dim='st_ocean')
drowned_temp_south = temp_south.ffill(dim='nxp').ffill(dim='st_ocean')
drowned_salt_south = salt_south.ffill(dim='nxp').ffill(dim='st_ocean')
drowned_ssh_south = ssh_south.ffill(dim='nxp')

drowned_v_west = v_west.ffill(dim='nyp').ffill(dim='st_ocean')
drowned_u_west =u_west.ffill(dim='nyp').ffill(dim='st_ocean')
drowned_temp_west = temp_west.ffill(dim='nyp').ffill(dim='st_ocean')
drowned_salt_west = salt_west.ffill(dim='nyp').ffill(dim='st_ocean')
drowned_ssh_west = ssh_west.ffill(dim='nyp')


In [10]:
n_southv_array = np.expand_dims(drowned_v_south.values,axis=2)
n_southu_array = np.expand_dims(drowned_u_south.values,axis=2)
n_southtemp_array = np.expand_dims(drowned_temp_south.values,axis=2)
n_southsalt_array = np.expand_dims(drowned_salt_south.values,axis=2)


In [11]:
n_southssh_array = np.expand_dims(ssh_south.values,axis=2)
n_finalsouthssh_array = np.swapaxes(n_southssh_array,-1,1)

In [51]:
n_westv_array = np.expand_dims(drowned_v_west.values,axis=3)
n_westu_array = np.expand_dims(drowned_u_west.values,axis=3)
n_westtemp_array = np.expand_dims(drowned_temp_west.values,axis=3)
n_westsalt_array = np.expand_dims(drowned_salt_west.values,axis=3)

n_westssh_array = np.expand_dims(ssh_west.values,axis=2)
#n_finalwestssh_array = np.swapaxes(n_westssh_array,-1,1)

(730, 58, 1, 301)

In [52]:
drowned_u_south

<xarray.DataArray (time: 730, st_ocean: 58, nxp: 301)>
dask.array<push, shape=(730, 58, 301), dtype=float32, chunksize=(515, 58, 301), chunktype=numpy.ndarray>
Coordinates:
  * st_ocean  (st_ocean) float64 0.5 1.5 2.5 3.5 ... 3.625e+03 4.3e+03 5e+03
  * time      (time) object 2012-01-01 12:00:00 ... 2013-12-31 12:00:00
    lon       (nxp) float64 77.0 77.07 77.15 77.22 ... 98.78 98.85 98.93 99.0
    lat       (nxp) float64 4.0 4.0 4.0 4.0 4.0 4.0 ... 4.0 4.0 4.0 4.0 4.0 4.0
Dimensions without coordinates: nxp
Attributes:
    regrid_method:  bilinear

In [53]:

south_final = xr.Dataset({
    "v" : (["time","zl","yq","xh"],n_southv_array),
"u" : (["time","zl","yq","xh"],n_southu_array),
"temp" : (["time","zl","yq","xh"],n_southtemp_array),
"salt" : (["time","zl","yq","xh"],n_southsalt_array),
"ssh" : (["time","yq","xh"],n_finalsouthssh_array)},
    coords = {"time":(["time",],drowned_v_south.time.values),
              "zl":(["zl",],drowned_v_south.st_ocean.values),
              "yq":(["yq",],[drowned_v_south.lat.values[0]]),
              "xh":(["xh"],drowned_v_south.lon.values)})

In [54]:
west_final = xr.Dataset({
    "v" : (["time","zl","yq","xh"],n_westv_array),
"u" : (["time","zl","yq","xh"],n_westu_array),
"temp" : (["time","zl","yq","xh"],n_westtemp_array),
"salt" : (["time","zl","yq","xh"],n_westsalt_array),
"ssh" : (["time","yq","xh"],n_westssh_array)},
    coords = {"time":(["time",],drowned_v_west.time.values),
              "zl":(["zl",],drowned_v_west.st_ocean.values),
              "yq":(["yq",],drowned_v_west.lat.values),
              "xh":(["xh"],[drowned_v_west.lon.values[0]])})

In [55]:
west_final

<xarray.Dataset>
Dimensions:  (time: 730, zl: 58, yq: 50, xh: 1)
Coordinates:
  * time     (time) object 2012-01-01 12:00:00 ... 2013-12-31 12:00:00
  * zl       (zl) float64 0.5 1.5 2.5 3.5 4.5 ... 3e+03 3.625e+03 4.3e+03 5e+03
  * yq       (yq) float64 4.0 4.076 4.152 4.228 ... 7.496 7.572 7.648 7.724
  * xh       (xh) float64 77.0
Data variables:
    v        (time, zl, yq, xh) float32 -0.2013 -0.2109 ... 0.02648 0.02648
    u        (time, zl, yq, xh) float32 -0.7064 -0.7727 ... -0.01114 -0.01114
    temp     (time, zl, yq, xh) float32 27.87 27.8 27.74 ... 1.717 1.717 1.717
    salt     (time, zl, yq, xh) float32 33.75 33.71 33.65 ... 34.74 34.74 34.74
    ssh      (time, yq, xh) float32 -0.03282 -0.02822 ... 0.06192 0.06206

In [56]:
caops_dir = "/home/nma/HDD_data/vinay/archives/IOMOM5Op/caops/"
sam_obc = xr.open_dataset(caops_dir+"obc_east.nc")
dz = sam_obc.dz_u_segment_002
dirv = "/home/nma/mom/MOM6dev/prepro/obc/gen_vgrid/"
IC_grid = xr.open_dataset(dirv+"2013_final.nc")
hy_depth = IC_grid.DEPTH.values
sam_obc.pt_segment_002

<xarray.DataArray 'pt_segment_002' (time: 34, zl: 40, yh: 192, xh: 1)>
[261120 values with dtype=float64]
Coordinates:
  * xh       (xh) float64 -77.04
  * yh       (yh) float64 18.09 18.17 18.24 18.32 ... 31.69 31.76 31.82 31.89
  * zl       (zl) float64 40.0 9.969e+36 9.969e+36 ... 9.969e+36 9.969e+36
  * time     (time) datetime64[ns] 2014-12-31T12:00:00 ... 2015-02-02T12:00:00
Attributes:
    units:    C

In [57]:
south_obc = south_final
west_obc = west_final
cur_depth = south_obc.zl.values

reg_south_obc = south_obc.interp(zl=hy_depth)
reg_west_obc = west_obc.interp(zl=hy_depth)


south_dz_new = np.array(len(reg_south_obc.xh.values)*[np.array(len(reg_south_obc.yq.values)*[np.array(len(reg_south_obc.time.values)*[np.insert(np.diff(hy_depth),
                                                                                      0,1)])])])


west_dz_new = np.array(len(reg_west_obc.xh.values)*[np.array(len(reg_west_obc.yq.values)*[np.array(len(reg_west_obc.time.values)*[np.insert(np.diff(hy_depth),
                                                                                      0,1)])])])




final_dz_south = south_dz_new.transpose(2,3,1,0)
final_dz_west = west_dz_new.transpose(2,3,1,0)


fin_reg_south = reg_south_obc.assign({"dz_v":(["time","zl","yq","xh"],final_dz_south),
                                     "dz_temp" : (["time","zl","yq","xh"],final_dz_south),
                                     "dz_salt" : (["time","zl","yq","xh"],final_dz_south),
                                     "dz_u" : (["time","zl","yq","xh"],final_dz_south)})

fin_reg_west = reg_west_obc.assign({"dz_v":(["time","zl","yq","xh"],final_dz_west),
                                     "dz_temp" : (["time","zl","yq","xh"],final_dz_west),
                                     "dz_salt" : (["time","zl","yq","xh"],final_dz_west),
                                     "dz_u" : (["time","zl","yq","xh"],final_dz_west)})

fin_reg_south.to_netcdf("/home/nma/mom/MOM6dev/prepro/obc/obcflather/nsouth.nc")
fin_reg_west.to_netcdf("/home/nma/mom/MOM6dev/prepro/obc/obcflather/nwest.nc")


In [58]:
fin_reg_west

<xarray.Dataset>
Dimensions:  (time: 730, zl: 40, yq: 50, xh: 1)
Coordinates:
  * time     (time) object 2012-01-01 12:00:00 ... 2013-12-31 12:00:00
  * yq       (yq) float64 4.0 4.076 4.152 4.228 ... 7.496 7.572 7.648 7.724
  * xh       (xh) float64 77.0
  * zl       (zl) float64 0.0 2.0 4.0 6.0 8.0 ... 2.5e+03 3e+03 4e+03 5e+03
Data variables:
    v        (time, zl, yq, xh) float64 nan nan nan ... 0.02648 0.02648 0.02648
    u        (time, zl, yq, xh) float64 nan nan nan ... -0.01114 -0.01114
    temp     (time, zl, yq, xh) float64 nan nan nan nan ... 1.717 1.717 1.717
    salt     (time, zl, yq, xh) float64 nan nan nan nan ... 34.74 34.74 34.74
    ssh      (time, yq, xh) float32 -0.03282 -0.02822 ... 0.06192 0.06206
    dz_v     (time, zl, yq, xh) float64 1.0 1.0 1.0 1.0 ... 1e+03 1e+03 1e+03
    dz_temp  (time, zl, yq, xh) float64 1.0 1.0 1.0 1.0 ... 1e+03 1e+03 1e+03
    dz_salt  (time, zl, yq, xh) float64 1.0 1.0 1.0 1.0 ... 1e+03 1e+03 1e+03
    dz_u     (time, zl, yq, xh) float64 1.0 1.0 1.0 1.0 ... 1e+03 1e+03 1e+03